<div style="text-align: center; padding: 30px">
  <h1><strong>Actividades con Conflicto</strong></h1>
  <h3><strong>Ejercicio 0</strong></h3>
</div>

El departamento de una universidad debe hacer la programación de los exámenes finales de los cursos que imparte dicho departamento. Cada examen tiene una duración de dos horas. Se necesita determinar el número de días necesario para la realización de dichos exámenes en los siguientes periodos de tiempo: 8:00 a 10:00, 10:15 a 12:15, 14:00 a 16:00 y 16:15 a 18:15, que dan como resultado cuatro periodos de dos horas por día, que deben servir para programar los exámenes de los cursos en distintos salones de clase. Para cada uno de los cursos, se proporciona una lista de los cursos que son incompatibles (ver Tabla 1), es decir, cursos que no se pueden programar al mismo tiempo, porque ambos están siendo por cursados por uno o varios de los estudiantes. Se requiere formular el problema para determinar el número mínimo de días necesarios para la realización de los exámenes de los 11 cursos. 

### **Uso de Python y el Solver SCIP**



En Python, utilizamos el solver `SCIP` para definir y resolver este problema. Para declarar variables binarias, se utiliza el método:

```python
solver.BoolVar(name)
```

In [1]:
from ortools.linear_solver import pywraplp

In [2]:
solver = pywraplp.Solver.CreateSolver('SCIP')

### **Conjuntos de Índices**

- $C$: Conjunto de cursos, $C = \{1, 2, \dots, 11\}$
- $P$: Conjunto de periodos, $P = \{1, 2, \dots, 11\}$
- $D$: Conjunto de días
- $I$: Conjunto de pares $(i, j)$ tales que $i, j \in C$ y están en conflicto.

### **Parámetros**

Los parámetros capturan la información del problema, que en este caso se refiere a los cursos en conflicto.

$$
\forall i, j \in C: 
a_{i,j} = 
\begin{cases} 
1  \text{si $i$ y $j$ están en conflicto}, \\ 
0   \text{en otro caso}.
\end{cases}
$$

In [3]:
# conjuntos
C = range(1, 12) # cursos
P = range(1, 5) # periodos
D = range(1, 12) # docentes

# conflictos/choques (ejemplo: curso 1 es incompatible con cursos 2, 5, 7, 10 y 11)
I = [(1, 2), (1, 5), (1, 7), (1, 10), (1, 11),
     (2, 1), (2, 5), (2, 7), (2, 10), (2, 11),
     (4, 3), (4, 5), (4, 6), (4, 7), (4, 10), (4, 11),
     (3, 4), (3, 5), (3, 6), (3, 7), (3, 9), (3, 10), (3, 11),
     (5, 1), (5, 2), (5, 3), (5, 4), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11),
     (6, 3), (6, 4), (6, 5), (6, 7), (6, 9), (6, 10), (6, 11),
     (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 8), (7, 9), (7, 10), (7, 11),
     (8, 5), (8, 7), (8, 10), (8, 11),
     (9, 3), (9, 5), (9, 6), (9, 7), (9, 10), (9, 11),
     (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9), (10, 11),
     (11, 1), (11, 2), (11, 3), (11, 4), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (11, 10) 
     ]

### **Variables de Decisión**

Las variables de decisión están relacionadas con la respuesta al problema.

$$
x_{i,j} = 
\begin{cases} 
1   \text{ si el examen del curso $i$ se asigna al periodo $j; \quad \forall i \in C, \forall j \in P$} \\ 
0   \text{ en otro caso}.
\end{cases}
$$


In [4]:
x = {} # creamos un diccionario para almacenar las variables 
for i in C: # cursos
    for d in D: # días
        for p in P: # periodos
            x[i, d, p] = solver.BoolVar(f'x[{i},{d},{p}]')  #nuestra binaria

$$
z_k = 
\begin{cases} 
1  \text{ si se utiliza el periodo $k; \quad \forall k \in P$} \\ 
0  \text{ en otro caso}.
\end{cases}
$$


In [5]:
z = {}
for d in D: # días
    z[d] = solver.BoolVar(f'z[{d}]') #variable binaria


$$
y = \text{ el número mínimo de días necesarios para programar los exámenes. }
$$


In [6]:
y = solver.IntVar(0, solver.infinity(), 'y')  # número mínimo de días necesarios
# IntVar es una variable entera desde 0 a infinito con nombre y

### **Función Objetivo**

El objetivo es minimizar el número mínimo de días necesarios para programar los exámenes'

$$
\min y
$$

In [7]:
solver.Minimize(y)

### **Restricciones**

Cada curso debe programarse exactamente una vez

$$
\sum_{d \in D} \sum_{p \in P} x_{i, d, p} = 1; \quad \forall i \in C
$$

In [8]:
for i in C: # cursos
    solver.Add(sum(x[i, d, p] for d in D for p in P) == 1) 
    # cada curso debe programarse exactamente (mínimo y máximo) una vez en algún día y periodo

Cursos incompatibles no pueden programarse al mismo tiempo

$$
x_{i, d, p} + x_{j, d, p} \leq 1; \quad \forall (i, j) \in I, \, \forall d \in D, \, \forall p \in P
$$

In [9]:
for (i, j) in I: # cursos incompatibles
    for d in D: # días
        for p in P: # periodos
            solver.Add(x[i, d, p] + x[j, d, p] <= 1) 

Si un periodo de un día se usa, $z_d = 1$


$$
\sum_{i \in C} \sum_{p \in P} x_{i, d, p} \leq |C| \cdot z_d; \quad \forall d \in D
$$

In [10]:
for d in D: # días 
    solver.Add(sum(x[i, d, p] for i in C for p in P) <= len(C) * z[d]) 


$$
y \geq \sum_{d \in D} z_d
$$

In [11]:
solver.Add(y >= sum(z[d] for d in D)) # número mínimo de días necesarios 

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000002183057DEF0> >

### **Resolver**

In [12]:
solver.Solve() 

0

In [13]:
print(f'Número mínimo de días: {solver.Objective().Value()}')

Número mínimo de días: 2.0


In [14]:
for i in C: # cursos
    for d in D: # días
        for p in P: # periodos
            if x[i, d, p].solution_value() == 1: # si el valor de la variable es 1
                print(f"Curso {i} en día {d} en periodo {p}") #calendario de exámenes

Curso 1 en día 1 en periodo 1
Curso 2 en día 1 en periodo 2
Curso 3 en día 1 en periodo 1
Curso 4 en día 1 en periodo 2
Curso 5 en día 1 en periodo 3
Curso 6 en día 1 en periodo 4
Curso 7 en día 2 en periodo 1
Curso 8 en día 1 en periodo 1
Curso 9 en día 1 en periodo 2
Curso 10 en día 2 en periodo 2
Curso 11 en día 2 en periodo 3
